# Dask Delayed

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask Delayed"
* https://docs.dask.org/en/latest/delayed.html
* Jesse C. Daniel. Data Science with Python and Dask.
* https://saturncloud.io/blog/a-data-scientist-s-guide-to-lazy-evaluation-with-dask/
* https://www.coiled.io/blog/how-to-learn-dask-in-2021

## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1\. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

In [1]:
import numpy as np
import dask.delayed as delayed
import dask
from time import sleep
import random

In [8]:
def cpu_task():
    numbers = [random.randint(0, 1000) for r_ in range(100_000)]
    return sum(numbers)
def io_task():
#     print('1')
    sleep(0.1)
#     print('2')

In [4]:
io_task()

1
2


In [6]:
%%time
r = [cpu_task() for _ in range(100)]

Wall time: 8.09 s


In [9]:
%%time
r = [io_task() for _ in range(100)]

Wall time: 11 s


In [10]:
cpu_task_delayed = dask.delayed(cpu_task)

In [11]:
cpu_task_delayed().compute()

50062174

In [12]:
%%time
r = [cpu_task_delayed() for _ in range(100)]
dask.compute(r)

Wall time: 8.54 s


([50052701,
  49793736,
  49966622,
  50065487,
  50017625,
  49826738,
  50006074,
  50069234,
  49939363,
  49941464,
  50127946,
  49944066,
  50047341,
  50054188,
  50175690,
  49918132,
  50072704,
  49933285,
  50067521,
  49968641,
  49905712,
  49971967,
  50061246,
  49995490,
  50101002,
  50062460,
  49826583,
  50009415,
  50214894,
  49838256,
  50051685,
  50013454,
  50068718,
  49987713,
  50120519,
  50113794,
  50089473,
  49905956,
  49922888,
  49924771,
  50116734,
  50063260,
  49950793,
  50041848,
  49934085,
  50233325,
  50041283,
  50087179,
  49942737,
  50087312,
  50018617,
  50036947,
  50143239,
  49949479,
  49770698,
  49828420,
  50055350,
  50089302,
  50017308,
  50000832,
  49794727,
  49844461,
  50068482,
  50222567,
  49961325,
  49990002,
  49976963,
  49926738,
  49987629,
  49917230,
  50022475,
  49952145,
  50146290,
  49889908,
  49890573,
  50205043,
  49985638,
  50097241,
  50036377,
  49859426,
  49974579,
  50133919,
  50118151,
  50

In [15]:
io_task_delayed = delayed(io_task)

In [16]:
%%time
r = [io_task_delayed() for _ in range(100)]
r = dask.compute(r, scheduler='multiprocessing')

Wall time: 3.78 s


In [17]:
%%time
r = [io_task_delayed() for _ in range(100)]
r = dask.compute(r, scheduler='threading')

Wall time: 2.77 s


## Лабораторная работа 14

In [1]:
from bs4 import BeautifulSoup
import numpy as np 
import re
import dask.delayed as delayed
import dask

1\. Напишите функцию, которая считывает файл формата xml из каталога `reviewers_full` и по данным этого файла формирует список словарей, содержащих следующие ключи: `id`, `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



In [2]:
keys = ['id', 'username', 'name', 'sex', 'country', 'mail', 'registered', 'birthdate', 'name_prefix', 'country_code']

In [15]:
def get_attrs(user):
    d = dict()
    for key in keys:
        attrs = re.compile(r'<\w+>(.*)<\/\w+>').findall(str(user.find(key)))
        if len(attrs) > 0:
            d[key] = attrs[0]
        else:
            d[key] = None
    return d

In [4]:
def xml_to_dict(file_number):
    with open(f'C:/Users/micha/Downloads/14_delayed_data/reviewers_full_{file_number}.xml') as f:
        reviews = BeautifulSoup(f, 'xml')
    review_dict = list(map(get_attrs, reviews.find_all('user')))
    return review_dict

2\. Измерьте время выполнения функции из задания 1 на всех файлах из каталога `reviewers_full`. Ускорьте время выполнения, используя `dask.delayed`.

In [42]:
%%time
list(map(xml_to_dict, np.arange(0, 20)));

`Wall time: 5min 38s`

In [6]:
xml_to_dict_delayed = delayed(xml_to_dict)

In [7]:
res = [xml_to_dict_delayed(file) for file in np.arange(0, 20)]

In [41]:
%%time
dask.compute(res, scheduler='multiprocessing');

`Wall time: 1min 44s`

3\. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

In [10]:
import dask.bag as db

In [8]:
reviewers = [xml_to_dict_delayed(file) for file in np.arange(0, 6)]

In [36]:
def info_change(user):
    user['id'] = int(user['id'])
    if user['birthdate'] != None:
        user['birthyear'] = str(user['birthdate'])[:4]
    else:
        user['birthyear'] = None
    return user

In [37]:
db_reviewers = db.from_delayed(reviewers).map(info_change)

In [38]:
db_reviewers.take(3)

({'id': 88005,
  'username': 'jacqueline00',
  'name': 'Michele Lewis',
  'sex': None,
  'country': None,
  'mail': 'morenocharlotte@yahoo.com',
  'registered': None,
  'birthdate': None,
  'name_prefix': None,
  'country_code': None,
  'birthyear': None},
 {'id': 68591,
  'username': 'daniellegomez',
  'name': None,
  'sex': 'F',
  'country': None,
  'mail': None,
  'registered': None,
  'birthdate': '2005-03-06',
  'name_prefix': None,
  'country_code': None,
  'birthyear': '2005'},
 {'id': 81003,
  'username': 'alucero',
  'name': 'Tammy Patton',
  'sex': None,
  'country': None,
  'mail': 'larsenrobert@gmail.com',
  'registered': None,
  'birthdate': None,
  'name_prefix': None,
  'country_code': None,
  'birthyear': None})

In [39]:
def filter_young(user):
    if user['birthyear'] != None:
        return user['birthyear'] > 1980
    else:
        return False

In [25]:
only_young_reviewers = db_reviewers.filter(filter_young)